In [15]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=getpass.getpass()

 ········


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki.name

'wikipedia'

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
# from langchain_fireworks import FireworksEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

In [7]:
vector_store=Chroma.from_documents(documents,OllamaEmbeddings())

In [8]:
retriever=vector_store.as_retriever()

In [9]:
from langchain.tools.retriever import create_retriever_tool

In [10]:
retriever_tool=create_retriever_tool(retriever,"langsmith_search","search for information about langsmith")

In [11]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [12]:
arxiv.name

'arxiv'

In [13]:
tools=[wiki,arxiv,retriever_tool]

In [37]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
    api_key="ollama",
    model="llama3",
    base_url="http://localhost:11434/v1",
)

In [38]:
# # from langchain_community.llms import LlamaCpp
# from langchain_community.chat_models import ChatLlamaCpp
# model=ChatLlamaCpp(model_path="./SanctumAI-meta-llama-3-8b-instruct.Q8_0.gguf")

In [23]:
from langchain import hub

prompt=hub.pull("vipintomar0/superb_system_instruction_prompt")

In [24]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [39]:
from langchain.agents import create_tool_calling_agent

agent=create_tool_calling_agent(model,tools,prompt)

In [40]:
# agent executor
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [41]:
agent_executor.invoke({"input":"tell me about langsmith"})



> Entering new AgentExecutor chain...
Langsmith is an unincorporated community located in Berkeley County, South Carolina, United States. The town has a long history dating back to the mid-19th century and is known for its small-town charm and rural way of life.

Here are some interesting facts about Langsmith:

1. **History**: Langsmith was established in the 1840s as a rural farming community. The town got its name from a local landowner, Lang Smith, who donated a large tract of land for the construction of a church.
2. **Location**: Langsmith is situated near the intersection of State Road S-15-14 and Lang Smith Road, about 25 miles west of Charleston, SC's city center.
3. **Community**: With a population of around 200 people, Langsmith is a small close-knit community where everyone knows each other. The town has a strong sense of tradition and cultural heritage.
4. **Agriculture**: Langsmith was historically a farming community, with many residents relying on agriculture for thei

{'input': 'tell me about langsmith',
 'output': "Langsmith is an unincorporated community located in Berkeley County, South Carolina, United States. The town has a long history dating back to the mid-19th century and is known for its small-town charm and rural way of life.\n\nHere are some interesting facts about Langsmith:\n\n1. **History**: Langsmith was established in the 1840s as a rural farming community. The town got its name from a local landowner, Lang Smith, who donated a large tract of land for the construction of a church.\n2. **Location**: Langsmith is situated near the intersection of State Road S-15-14 and Lang Smith Road, about 25 miles west of Charleston, SC's city center.\n3. **Community**: With a population of around 200 people, Langsmith is a small close-knit community where everyone knows each other. The town has a strong sense of tradition and cultural heritage.\n4. **Agriculture**: Langsmith was historically a farming community, with many residents relying on agri

In [42]:
agent_executor.invoke({"input":"tell me about machine learning"})



> Entering new AgentExecutor chain...
Machine learning!

Machine learning is a subset of artificial intelligence (AI) that enables computers to learn from data without being explicitly programmed. In other words, machines can discover patterns, relationships, and make predictions or decisions based on the data they receive.

Here's a breakdown:

**How it works:**

1. **Data collection**: You gather a dataset related to the problem you want to solve (e.g., images, text, sensor readings).
2. **Model training**: The machine learning algorithm analyzes the data and identifies patterns, relationships, or underlying structures.
3. **Training**: The algorithm adjusts its parameters or weights based on the insights gained from the data. This process is called "training" the model.
4. **Validation**: You evaluate the performance of the trained model using a separate dataset (the "test set").
5. **Deployment**: Once the model is validated, you can use it to make predictions, classify new data,

{'input': 'tell me about machine learning',
 'output': 'Machine learning!\n\nMachine learning is a subset of artificial intelligence (AI) that enables computers to learn from data without being explicitly programmed. In other words, machines can discover patterns, relationships, and make predictions or decisions based on the data they receive.\n\nHere\'s a breakdown:\n\n**How it works:**\n\n1. **Data collection**: You gather a dataset related to the problem you want to solve (e.g., images, text, sensor readings).\n2. **Model training**: The machine learning algorithm analyzes the data and identifies patterns, relationships, or underlying structures.\n3. **Training**: The algorithm adjusts its parameters or weights based on the insights gained from the data. This process is called "training" the model.\n4. **Validation**: You evaluate the performance of the trained model using a separate dataset (the "test set").\n5. **Deployment**: Once the model is validated, you can use it to make pr